In [1]:
!pip install langchain-huggingface
!pip install langchain-community
!pip install faiss-cpu
!pip install huggingface_hub
!pip install langchain_community pypdf

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader

import requests

print("Start..")

# load raw document file
textbook_url = "https://web.eecs.utk.edu/~bnd674/MedLLM/content/procedures.txt"
report_url = "https://web.eecs.utk.edu/~bnd674/MedLLM/content/489_gemini_sum.csv"

# Local file paths
textbook_file = "D:\\Programs\\489-594_MedLLM\\Data\\procedures.txt"
report_file = "D:\\Programs\\489-594_MedLLM\\Data\\489_gemini_sum.csv"

# Download the text file
#with open(textbook_file, "wb") as f:
#    f.write(requests.get(textbook_url).content)

# Download the CSV file
#with open(report_file, "wb") as f:
#    f.write(requests.get(report_url).content)



textbook_loader = TextLoader(textbook_file, encoding='utf8')
report_loader = CSVLoader(report_file, source_column ="SUMMARY")

textbook_docs = textbook_loader.load()
report_docs = report_loader.load()

summary_docs = [Document(doc.metadata['source']) for doc in report_docs]



Start..


# Split documents and label with metadata


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter



text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,separators=['NEW SECTION:'])
text_splits = text_splitter.split_documents(textbook_docs)

report_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=['New Patient:'])
report_splits = report_splitter.split_documents(summary_docs)

# label docs with metadata
for doc in text_splits:
  doc.metadata = {'source': 'Textbook'}
  doc.page_content = doc.page_content[13:]

for doc in report_splits:
  doc.metadata = {'source': 'Reports'}
  doc.page_content = doc.page_content[13:]

splits = text_splits + report_splits



# Load embedding model and vector store

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
import faiss
from uuid import uuid4


# load embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
embedding_dim = len(embedding_model.embed_query("hello world"))



# Initialize vector store
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embedding_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

ids = [i for i in range(len(splits))]


vector_store.add_documents(documents=splits, ids=ids)


/home/tlsto/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75]

In [5]:
retriever = vector_store.as_retriever(search_kwargs={"k": 1})
textbook_retriever = vector_store.as_retriever(search_kwargs={"k":5})
report_retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [6]:
print(type(textbook_retriever))
print(type(report_retriever))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>
<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [7]:
# Save vector store if needed

vector_store.save_local("FAISS_vector_store")

In [8]:
from langchain_core.retrievers import BaseRetriever
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.documents import Document

class MyCustomRetriever(BaseRetriever):
    textbook_retriever: VectorStoreRetriever
    report_retriever: VectorStoreRetriever

    def get_relevant_documents(self, query: str) -> list[Document]:

        textbook_docs = self.textbook_retriever.invoke(
            query, filter={"source": "Textbook"}
        )


        cat_reports = query + "\n"
        for doc in textbook_docs:
            cat_reports += doc.page_content + "\n"


        report_docs = self.report_retriever.invoke(
            cat_reports, filter={"source": "Reports"}
        )


        return textbook_docs + report_docs


/tmp/ipykernel_11571/412335110.py:5: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class MyCustomRetriever(BaseRetriever):


In [9]:
#from google.colab import userdata
#import google.generativeai as genai
from langchain.llms import HuggingFaceHub
from langchain_huggingface import HuggingFacePipeline
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
#model_id = "unsloth/Llama-3.2-1B"

print(f"using model: {model_id}")
#api_key = userdata.get('huggingface')
#login(api_key)
tokenizer = AutoTokenizer.from_pretrained(model_id)
pipe = pipeline("text-generation", model = model_id, tokenizer=tokenizer, max_new_tokens = 2048)
llm = HuggingFacePipeline(pipeline=pipe)

using model: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Device set to use cuda:0


In [10]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

retriever = MyCustomRetriever(textbook_retriever=textbook_retriever, report_retriever=report_retriever)

# system_prompt = (
#   """You are a medical assistant helping people treat injuries.
#   Use the context below to provide a clear, step-by-step treatment plan, including medications, procedures, and follow-up care.
#   \n\n
#   ### Context ###
#   {context}
#   ### End Context ###
# """
# )
# prompt = ChatPromptTemplate.from_messages([
#     ("system", system_prompt),
#     ("human", "{input}")
# ])
system_prompt = (
    """You are a knowledgeable and calm medical assistant specializing in trauma and wound care.
Based on the ER medical records below, help the user understand how to treat their injury on their own or prepare it until professional care is available.

### Context ###
{context}

### Question ###
{input}

### Instructions ###
Give a calm, step-by-step treatment plan specifically for this type of injury.
Include wound care, medications, and what procedures need to be done.

### Guidelines ###
- Provide a clear, step-by-step care plan tailored to the specific injury.
- Include any necessary wound care, over-the-counter medications, or procedures.
- Note when emergency care should be sought.
- Keep your tone reassuring, practical, and easy to understand.


### Answer ###
"""
    
)


USER_QUESTION = "I shot my hand with a nail gun and the nail is still lodged in my hand. What do I do?"

prompt = ChatPromptTemplate.from_template(system_prompt)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [11]:
response = llm.invoke(USER_QUESTION)
#print("RAW RESPONSE:\n", response)
print(response)

I shot my hand with a nail gun and the nail is still lodged in my hand. What do I do? I have a medical condition, but I'm not sure what it is. I don't know if I can perform certain actions, like eating, drinking, or working. I need to figure out what to do next.

I have a nail gun, which is a weapon, but I'm not sure if it's legal to use in my country. I need to check the laws. I also need to find out what my medical condition is. Maybe it's something like diabetes or high blood pressure.

I'm worried about the consequences of not addressing my condition. I want to make sure I don't get stuck in a situation where I can't do anything. I need to figure out a way to manage my condition and regain control of my hands.

I think I can't just stop the nail, but maybe I can somehow release it so I can perform actions. However, I'm not sure how to do that. Maybe I can use some kind of device or tool to release it.

I also need to make sure I don't take any unnecessary risks. I don't want to hur

In [12]:
results = rag_chain.invoke({"input": USER_QUESTION})

#print(results)
print(results['answer'].split('### End Context ###'))

["Human: You are a knowledgeable and calm medical assistant specializing in trauma and wound care.\nBased on the ER medical records below, help the user understand how to treat their injury on their own or prepare it until professional care is available.\n\n### Context ###\nFUNCTIONAL_ANATOMY – ANTERIOR INTEROSSEOUS NERVE INJURY\n\nProcedure: Use the “OK sign” test. Ask the patient to form a circle with the thumb and index finger.\n\nFindings: Flattening of the distal thumb and index interphalangeal joints indicates weakness in flexor pollicis longus and flexor digitorum profundus, due to AIN palsy.\n\nNEW SECTION: FUNCTIONAL_ANATOMY – ULNAR NEUROPATHY\n\nProcedure: Electrodiagnostic testing for denervation in intrinsic hand muscles (e.g., abductor digiti minimi, interossei, lumbricals to digits 4 and 5).\n\nAnatomic note: Compression commonly occurs at the retrocondylar groove or cubital tunnel. The flexor carpi ulnaris is usually spared.\n\nVascular_Diseases – DIABETIC WOUNDS (DIAGNO

In [13]:
# answer = results['answer'].split('### End Context ###')
print(results['answer'])
answer = results['answer'].split('</think>')[1]

print(len(answer))


print(f"Question: \n{USER_QUESTION}\nResponse: ")
print(answer)

Human: You are a knowledgeable and calm medical assistant specializing in trauma and wound care.
Based on the ER medical records below, help the user understand how to treat their injury on their own or prepare it until professional care is available.

### Context ###
FUNCTIONAL_ANATOMY – ANTERIOR INTEROSSEOUS NERVE INJURY

Procedure: Use the “OK sign” test. Ask the patient to form a circle with the thumb and index finger.

Findings: Flattening of the distal thumb and index interphalangeal joints indicates weakness in flexor pollicis longus and flexor digitorum profundus, due to AIN palsy.

NEW SECTION: FUNCTIONAL_ANATOMY – ULNAR NEUROPATHY

Procedure: Electrodiagnostic testing for denervation in intrinsic hand muscles (e.g., abductor digiti minimi, interossei, lumbricals to digits 4 and 5).

Anatomic note: Compression commonly occurs at the retrocondylar groove or cubital tunnel. The flexor carpi ulnaris is usually spared.

Vascular_Diseases – DIABETIC WOUNDS (DIAGNOSTIC & TREATMENT P

In [ ]:
print("[MedLLM] INTERACTIVE MODE STARTED")
while True:
    USER_QUESTION = input("\n\n\n[MedLLM] >>: ")
    print("\n[MedLLM] Thinking... (this could take a minute)")
    response = llm.invoke(USER_QUESTION)
    print("\n[MedLLM (1/2)] llm done...")
    results = rag_chain.invoke({"input": USER_QUESTION})
    print("\n[MedLLM (2/2)] rag done...")
    answer = results['answer'].split('</think>')[1]
    print(f"Question: \n{USER_QUESTION}\nResponse: ")
    print(answer)

[MedLLM] INTERACTIVE MODE STARTED





[MedLLM] >>:  i have a nosebleed



[MedLLM] Thinking... (this could take a minute)

[MedLLM (1/2)] llm done...

[MedLLM (2/2)] rag done...
Question: 
i have a nosebleed
Response: 


**Step-by-Step Treatment Plan for a Nosebleed:**

1. **Assessment and Evaluation:**
   - Perform a CT scan to assess the extent of the bleeding and any underlying conditions like hemodynamic instability.

2. **Wound Care:**
   - Apply a layer of ice packs to the bleeding area to reduce swelling and promote healing.
   - Use an anesthetic (e.g., ice packs) to numb the area temporarily.

3. **Medication:**
   - Administer oral antipyretics (e.g., bicocaine) for pain and inflammation.
   - Consider over-the-counter antihistamines if the bleeding is severe.

4. **Wound Management:**
   - Use a splint or bandage to immobilize the area and prevent further swelling.
   - Apply pressure to promote healing.

5. **Monitoring:**
   - Keep an eye on the bleeding for signs of progression like swelling or edema.

6. **Monitoring for Intraembolic Fluid:**